# Retrieving CYGNSS Soil Moisture data

Data Product:
* Level 3 - daily gridded

Documentation: https://podaac.jpl.nasa.gov/dataset/CYGNSS_L3_SOIL_MOISTURE_V3.2

## Setup

In [ ]:
import earthaccess
import pandas
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
earthaccess.login()

## Retrieve

In [ ]:
results = earthaccess.search_data(
    short_name='CYGNSS_L3_SOIL_MOISTURE_V3.2',
    temporal=('2024-01-01','2024-01-03'),
    # bounding_box=(-135.0, -38.15, 164.0, 38.15),
    cloud_hosted=True
)

In [ ]:
results

In [ ]:
files = earthaccess.download(results[0:5], local_path='../data/cygnss_sm')

## Download, Read and Explore

High res 9x9 and low res 36x36 returned in the separate granules from same function call.

In [ ]:
print(files[2])
print(files[3])

In [ ]:
fname = files[3]
print(f"Reading file: {fname}")
ds = xr.open_dataset(fname)

In [ ]:

sm = ds["SM_daily"].isel(time=0)
tstr = str(sm["time"].values)

plt.figure(figsize=(8,6))
im = plt.imshow(sm, origin="lower")

cbar = plt.colorbar(im, location="right", fraction=0.016, pad=0.04)
cbar.set_label("Soil Moisture")

plt.title(f"CYGNSS SM_daily — {tstr}")
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.savefig("../outputs/cygnss_eg.png", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
sm = ds["SM_subdaily"].isel(timeslices=0)
# tstr = str(sm["timeslices"].values)

plt.figure(figsize=(8,6))
im = plt.imshow(sm, origin="lower")

cbar = plt.colorbar(im, location="right", fraction=0.016, pad=0.04)
cbar.set_label("Soil Moisture")

plt.title(f"CYGNSS SM_subdaily — {tstr}")
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.savefig("../outputs/cygnss_eg.png", bbox_inches="tight", pad_inches=0)
plt.show()

## Read without Downloading

In [ ]:
filenames = [r.get("umm").get("GranuleUR") for r in results]
keep = ["9km" in fi for fi in filenames]

fileobjs = earthaccess.open([res for res, k in zip(results, keep) if k])

In [ ]:
ds1 = xr.open_dataset(fileobjs[0])

In [ ]:
ds = xr.open_mfdataset(fileobjs, combine="by_coords")